In [1]:
import os
import xarray as xr
import geopandas as gpd
import rioxarray  # important: enables .rio

# ========= 1. PATHS =========
input_folder = r"Data\Climate Data\ET"            # your input .nc folder
output_folder = r"Data\Climate Data\2. Tasmania\ET_tas"   # output folder
shapefile_path = r"Data\Tasmania_Boundary\tasmania_boundary.shp"  # <-- update

os.makedirs(output_folder, exist_ok=True)

# ========= 2. READ SHAPEFILE =========
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs("EPSG:4326")
shapes = gdf.geometry.values

# ========= 3. LOOP THROUGH FILES =========
for fname in os.listdir(input_folder):
    if not fname.endswith(".nc"):
        continue

    in_path = os.path.join(input_folder, fname)
    base = fname[:-3]  # strip ".nc"
    out_path = os.path.join(output_folder, f"{base}_cropped.nc")

    print(f"Processing: {fname}")

    # Open dataset
    ds = xr.open_dataset(in_path)

    # ---- Tell rioxarray which dims are spatial ----
    # ABARES grid uses dims 'lon' and 'lat'
    # If your file uses something slightly different, print(ds.dims) to check.
    ds = ds.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=False)

    # Ensure CRS is set
    if ds.rio.crs is None:
        ds = ds.rio.write_crs("EPSG:4326")

    # ---- Clip ----
    try:
        ds_clipped = ds.rio.clip(
            shapes,
            ds.rio.crs,
            drop=True,
            from_disk=True
        )
    except Exception as e:
        print(f"  ⚠ Error clipping {fname}: {e}")
        ds.close()
        continue

    # Save clipped dataset
    ds_clipped.to_netcdf(out_path)

    ds.close()
    ds_clipped.close()

    print(f"  ✔ Saved: {out_path}")

print("All files clipped and saved with _cropped suffix.")


Processing: 1991.et_short_crop.nc
  ✔ Saved: Data\Climate Data\2. Tasmania\ET_tas\1991.et_short_crop_cropped.nc
Processing: 1992.et_short_crop.nc
  ✔ Saved: Data\Climate Data\2. Tasmania\ET_tas\1992.et_short_crop_cropped.nc
Processing: 1993.et_short_crop.nc
  ✔ Saved: Data\Climate Data\2. Tasmania\ET_tas\1993.et_short_crop_cropped.nc
Processing: 1994.et_short_crop.nc
  ✔ Saved: Data\Climate Data\2. Tasmania\ET_tas\1994.et_short_crop_cropped.nc
Processing: 1995.et_short_crop.nc
  ✔ Saved: Data\Climate Data\2. Tasmania\ET_tas\1995.et_short_crop_cropped.nc
Processing: 1996.et_short_crop.nc
  ✔ Saved: Data\Climate Data\2. Tasmania\ET_tas\1996.et_short_crop_cropped.nc
Processing: 1997.et_short_crop.nc
  ✔ Saved: Data\Climate Data\2. Tasmania\ET_tas\1997.et_short_crop_cropped.nc
Processing: 1998.et_short_crop.nc
  ✔ Saved: Data\Climate Data\2. Tasmania\ET_tas\1998.et_short_crop_cropped.nc
Processing: 1999.et_short_crop.nc
  ✔ Saved: Data\Climate Data\2. Tasmania\ET_tas\1999.et_short_crop_cro